<a href="https://colab.research.google.com/github/UN-GCPDS/curso_IA_CHEC/blob/main/Modulo7/Notebooks/LanchainNeo4j3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LIBRERIAS

In [ ]:
%%capture
!pip install langchain-community
!pip install langchain

In [ ]:
%%capture
!pip install neo4j
!pip install openai

In [ ]:
%%capture
!pip install httpx==0.27.2

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

## CONEXIÓN NEO4J

In [ ]:
os.environ['OPENAI_API_KEY'] = "API-KEY"

graph = Neo4jGraph(
    url="URL",
    username="USERNAME",
    password="PASSWORD"
)

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True,allow_dangerous_requests=True
)

<ipython-input-5-52b629513bc9>:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  ChatOpenAI(temperature=0), graph=graph, verbose=True,allow_dangerous_requests=True


In [ ]:
chain.run("""
Which fortnite streamer has the most followers?
""")

<ipython-input-6-7998b9f81cbd>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("""




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:Stream)-[:PLAYS]->(:Game {name: "Fortnite"})
RETURN s.name, s.followers
ORDER BY s.followers DESC
LIMIT 1;
Full Context:
[{'s.name': 'thegrefg', 's.followers': 7269018}]

> Finished chain.


'thegrefg has 7,269,018 followers.'

In [ ]:
chain.run("""
Which italian streamer has the most followers?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:Stream)-[:HAS_LANGUAGE]->(:Language{name: "Italian"})
RETURN s.name, s.followers
ORDER BY s.followers DESC
LIMIT 1
Full Context:
[]

> Finished chain.


"I don't know the answer."

## EJEMPLO QUERY BASICA

In [ ]:
from langchain.prompts.prompt import PromptTemplate

In [ ]:
 CYPHER_GENERATION_TEMPLATE = """
Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Cypher examples:
# How many streamers are from Norway?
MATCH (s:Stream)-[:HAS_LANGUAGE]->(:Language {{name: 'no'}})
RETURN count(s) AS streamers

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

The question is:
{question}"""
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

In [ ]:
chain_language_example = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,allow_dangerous_requests=True
)

chain_language_example.run("""
Which Itallian streamer has the most followers?
""")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:Stream)-[:HAS_LANGUAGE]->(:Language {name: 'it'})
RETURN s.name AS streamer, s.followers AS followers
ORDER BY followers DESC
LIMIT 1
Full Context:
[{'streamer': 'pow3rtv', 'followers': 1530428}]

> Finished chain.


'pow3rtv has 1,530,428 followers.'